In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 https://developer.download.nvidia.com/compute/c

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-04 18:24:37--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.00MB/s    in 0.2s    

2021-12-04 18:24:37 (6.00 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

In [6]:
# Create the customers_table DataFrame
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
from pyspark.sql.functions import count
customers_df = df.groupby("customer_id")\
                .agg({"*": "count"})\
                .withColumnRenamed("count(1)", "customer_count")
customers_df.show(20)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   46909180|             6|
|   42560427|             7|
|   43789873|             3|
|   22037526|             2|
|   34220092|             2|
|   42801586|             1|
|    9565734|             2|
|   15829398|             1|
|   38247118|             1|
|   32478248|             2|
|   48114630|             1|
|   23085063|             1|
|   32787070|             3|
|   43515569|             1|
|    4919528|             2|
|    5088547|             2|
|   41852407|             3|
|   49703087|             1|
|   12713799|             1|
|   36728141|             8|
+-----------+--------------+
only showing top 20 rows



In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(['product_id', 'product_title']).drop_duplicates()
products_df.show(20)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B010LVPJH6|LG G Watch Urbane...|
|B00K5ZNXZ4|Minisuit Sporty A...|
|B00S9RBQOK|iPhone 6 Plus Cas...|
|B0116N7GYC|Selfie Stick, Por...|
|B009UNH0CY|Plantronics Voyag...|
|B00L8GFYAG|Eallc New Quality...|
|B00R3LMTI0|Kaleidio [Wallop ...|
|B011R0VG36|Galaxy Note 4 Cas...|
|B00BXX0QVQ|iKross Black Dual...|
|B00F4AYI2M|Incipio DualPro C...|
|B00GPI3OHC|Retevis H-777 2 W...|
|B00Y9ZUVU6|Tiwkich 2 in 1 Du...|
|B00W65SYHS|LG G4 case, Caseo...|
|B00V5OU6CW|S5 Leather case,P...|
|B00LP3FSH6|Escort Coiled Sma...|
|B00MIO2KRC|Black Box G1W-C C...|
|B00V5FZM0M|   KoKo Cases 5/5S !|
|B00T1KO2TA|iPhone 6 & 6S Cas...|
|B00YU9XOTQ|Galaxy S5 Screen ...|
|B00PI7IGHE|Soyan Latest DZ09...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create the customers_table DataFrame
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
from pyspark.sql.functions import count
customers_df = df.groupby("customer_id")\
                .agg({"*": "count"})\
                .withColumnRenamed("count(1)", "customer_count")
customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   46909180|             6|
|   42560427|             7|
|   43789873|             3|
|   22037526|             2|
|   34220092|             2|
+-----------+--------------+
only showing top 5 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
review_id_df = df.select(['review_id','customer_id', 'product_id', 'product_parent']).drop_duplicates()
review_id_df.show(20)

+--------------+-----------+----------+--------------+
|     review_id|customer_id|product_id|product_parent|
+--------------+-----------+----------+--------------+
|R1006VH7CWQ7L2|   18974232|B004TDS29A|     572176733|
|R10084NQPMAE8T|    2157505|B00JVOC9X2|     971969686|
|R1008M23N90ABA|   11128745|B00IMMO8LA|     730409480|
|R100BBEWKEA65E|   19917115|B00730HR24|     230623722|
|R100FCX761Y5M5|   13149242|B00NHL4QKO|     314056284|
|R100U0AKSH9NIR|   37733169|B00P682U7M|     178250058|
|R100UBK2FXVISD|   51988885|B0075IQKYA|     138685242|
|R100UUZN26I8O4|   44552892|B003SHQXRG|     774327427|
|R100UVH1DG0Y49|   41948600|B00D4YJXW0|      11746350|
|R1014Z60YEPUM7|   28068239|B004RBL51Q|     165875437|
|R10151TU03KLKA|   41596510|B005NF5NTK|     908777325|
|R1016VTAKHBOPD|   33132401|B004AYIPZ4|      86413153|
|R1017LKPAVQTRS|   25214622|B009O58D8S|     993902459|
|R101HPI5NFNS81|   16772286|B0036FS1H0|     844766791|
|R101K82V2AA9RZ|   18900285|B003XDU8OO|     486328069|
|R101PU8SO

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|                Y|
|R15V54KBMTQWAY|          4|            0|          0|   N|                N|
| RY8I449HNXSVF|          5|            0|          0|   N|                Y|
|R18TLJYCKJFLSR|          5|            0|          0|   N|                Y|
|R1NK26SWS53B8Q|          5|            0|          0|   N|                Y|
|R11LOHEDYJALTN|          5|            0|          0|   N|                Y|
|R3ALQVQB2P9LA7|          4|            0|          0|   N|                Y|
|R3MWLXLNO21PDQ|          5|            0|          0|   N|                Y|
|R2L15IS24CX0LI|          5|            0|          0|   N|                Y|
|R1DJ8976WPWVZU|          3|            0|          0|   N|     

In [18]:
# Bias Staatistics

In [20]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
total_votes_df = vine_df.filter("total_votes > 20")
total_votes_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2DHMT741HV645|          5|           30|         35|   N|                Y|
|R1D17BEB2785AD|          5|           16|         23|   N|                Y|
|R1QPIOS7YKHDPN|          5|           24|         26|   N|                Y|
|R1MXAIYHLXIFZW|          1|           27|         29|   N|                Y|
| R8TE546KUXF4M|          5|           64|         67|   N|                Y|
| R6F4QVUTY4HU7|          5|           23|         24|   N|                Y|
|R36D1KS3CQ47PN|          4|          177|        199|   N|                N|
| RR0XRV40MGB57|          5|           19|         24|   N|                Y|
|R1E1QK0XG8MOXH|          5|           25|         26|   N|                Y|
|R3SMIOLIMDKSWI|          1|           46|         48|   N|     

In [21]:
# new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_votes_df = total_votes_gt_20_df.filter("(helpful_votes / total_votes) > 0.5")
helpful_votes_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2DHMT741HV645|          5|           30|         35|   N|                Y|
|R1D17BEB2785AD|          5|           16|         23|   N|                Y|
|R1QPIOS7YKHDPN|          5|           24|         26|   N|                Y|
|R1MXAIYHLXIFZW|          1|           27|         29|   N|                Y|
| R8TE546KUXF4M|          5|           64|         67|   N|                Y|
| R6F4QVUTY4HU7|          5|           23|         24|   N|                Y|
|R36D1KS3CQ47PN|          4|          177|        199|   N|                N|
| RR0XRV40MGB57|          5|           19|         24|   N|                Y|
|R1E1QK0XG8MOXH|          5|           25|         26|   N|                Y|
|R3SMIOLIMDKSWI|          1|           46|         48|   N|     

In [23]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
vine_votes_df = helpful_votes_df.filter("vine == 'Y'")
vine_votes_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1LPLEZ5ST999T|          4|          261|        272|   Y|                N|
|R2E5UC1L37RSZG|          5|          215|        224|   Y|                Y|
|R1GA5ALM462AZY|          5|           23|         26|   Y|                N|
|R2TMTT0VL8RBNW|          1|           17|         22|   Y|                N|
| RKL4ZK5EY5FAN|          4|           34|         37|   Y|                N|
|R3AD9GXP6HYU0S|          1|           22|         25|   Y|                N|
| RQS47OP8V796C|          5|           21|         24|   Y|                N|
| RJ99L15QR4XSD|          4|           20|         23|   Y|                N|
|R258XYEX1U1YEI|          3|          203|        217|   Y|                N|
|R3NL12G8N78O60|          5|           33|         36|   Y|     

In [28]:
# Filter the DataFrame or table created in Step 3, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (unpaid), vine == 'N'.
not_vine_votes_df = helpful_votes_df.filter("vine == 'N'")
not_vine_votes_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2DHMT741HV645|          5|           30|         35|   N|                Y|
|R1D17BEB2785AD|          5|           16|         23|   N|                Y|
|R1QPIOS7YKHDPN|          5|           24|         26|   N|                Y|
|R1MXAIYHLXIFZW|          1|           27|         29|   N|                Y|
|R2UIZP00RWN5ID|          5|           47|         49|   N|                Y|
| R8TE546KUXF4M|          5|           64|         67|   N|                Y|
| R6F4QVUTY4HU7|          5|           23|         24|   N|                Y|
|R36D1KS3CQ47PN|          4|          177|        199|   N|                N|
| RR0XRV40MGB57|          5|           19|         24|   N|                Y|
|R1E1QK0XG8MOXH|          5|           25|         26|   N|     

In [32]:
num_vines = vine_votes_df.count()

In [35]:
num_other = not_vine_votes_df.count()

In [30]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
fivestar_reviews = vine_votes_df.filter("star_rating == 5").count()


In [48]:
fivestar_other = not_vine_votes_df.filter("star_rating < 5").count()

In [49]:
fivestar_percentage = round((fivestar_reviews / num_vines *100),2)

In [50]:
other_percentage = round((fivestar_other/ num_other * 100), 2)

In [51]:
print(f"Vines: reviews={num_vines}, 5-star={fivestar_reviews} ({fivestar_percentage} %)")

Vines: reviews=60989, 5-star=28826 (47.26 %)


In [52]:
print(f"Other: reviews={num_other}, 5-star={fivestar_other} ({other_percentage} %)")

Other: reviews=60989, 5-star=32163 (52.74 %)
